In [ ]:
# -*- coding: utf-8 -*-
"""
@Author  : rabbit
@File    : discord-chat-bot.py
@Time    : 8/4/24 12:34 PM
@desc    : discord自动聊天
"""
import requests
import json
import random
import time
import traceback

""" ************ 配置项  开始  ************ """
# 聊天的时间间隔（10-30s，可自行修改）
time_min = 120
time_max = 300
# 把你想让机器人说的内容写在这里，(随机发送)单引号里写内容，逗号隔开
globals_message_list = [""]
# 支持多个频道 id
channels_id_list = [""]
# 支持多个 authorization，意味着支持多账号
authorization_list = [""]
# 是否走代理？True为走代理，False不走
is_proxy = False
# 本地代理设置
proxies = {"http": "http://127.0.0.1:41091", "https": "https://127.0.0.1:41091"}
""" ************ 配置项  结束  ************ """
msg_dict = {}


def chat(channels, authorizations):
    for authorization in authorizations:
        header = {
            "Authorization": authorization,
            "Content-Type": "application/json",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        }
        for channel_id in channels:
            # nonce 一共 18 位数字
            nonce = channel_id[:4] + "".join(random.choice("0123456789") for _ in range(14))
            msg = {
                "content": get_context(),
                "nonce": nonce,
                "tts": False,
            }
            message_url = "https://discord.com/api/v9/channels/{}/messages".format(channel_id)
            try:
                if is_proxy:
                    res = requests.post(url=message_url, headers=header, data=json.dumps(msg), proxies=proxies)
                else:
                    res = requests.post(url=message_url, headers=header, data=json.dumps(msg), timeout=10)
                print(res.content)
                print("发送成功！！" + json.dumps(msg))
            except:
                traceback.print_exc()
                continue
        time.sleep(random.randrange(1, 3))


def get_context():
    # 默认为这三个消息内容
    message_list = [
        "你好",
        "GM",
        "Good Project！",
    ]
    # 若用户自定义了消息，赋值返回
    if globals_message_list:
        message_list = globals_message_list
    context = random.choice(message_list)
    if context not in msg_dict.keys():
        msg_dict[context] = 1
    else:
        msg_dict[context] += 1
    count = msg_dict[context]
    # 若出现的语言已经发过一次，在发就示意+1
    #if count > 1:
    #    context = context + "+1"
    # 若发送消息已经有一半随机到了，清空字典
    if len(msg_dict) == len(globals_message_list) / 2:
        msg_dict.clear()
    # 随机返回
    return context


def run():
    if not channels_id_list or not authorization_list:
        print("请完善配置后，在开始运行！")
        return
    while True:
        try:
            chat(channels_id_list, authorization_list)
            sleep_time = random.randrange(time_min, time_max + 1)  # 随机休息间隔，单位s
            time.sleep(sleep_time)
        except:
            traceback.print_exc()


if __name__ == "__main__":
    run()  # 执行
